# Merged basal zonal stats colation

This notebook looks for zonal stats subdirectories and the zonal stats csv outputs within them and concatenates all files into one data frame which is exported as a csv to the output directory.

The following conditions apply:

 - run after Seasonal Biomass Zonal Pipeline.
 - env = base

In [1]:
# Dictionary identifies the data structure of the reference image

dict_ = {"dbi_zonal_stats": "seasonal", "dim_zonal_stats": "seasonal",
"dis_zonal_stats": "seasonal",
"dja_zonal_stats": "seasonal",
"dka_zonal_stats": "annual",
"dp0_zonal_stats": "unknown",
"fpc_zonal_stats": "single",
"fpca2_zonal_stats": "seasonal",
"h99a2_zonal_stats": "seasonal",
"pg_zonal_stats": "seasonal",
"ref_zonal_stats": "single",
"stc_zonal_stats": "seasonal",
"th_zonal_stats": "seasonal"
}

In [2]:
import pandas as pd
from glob import glob
import os
from calendar import monthrange
from datetime import datetime

In [3]:
basal = r"F:\cdu\data\output\c_bio_site_totals_v4_edit.csv"
basal_df = pd.read_csv(basal)

In [4]:
dir_ = r"F:\cdu\data\zonal_stats\zonal_stats_20220921\rmcgr_20220920_1303"
output_dir = r"F:\cdu\data\output\zonal_stats"

In [5]:
def start_seasonal_date(date_):
    """ extract the end dates of the seasonal image zonal stats."""
    
    year = date_[:4]
    month = date_[4:]
    
    start_date = str(year) + str(month) + "01"
    
    return start_date
    

def end_seasonal_date(date_):
    
    """ extract the start dates of the seasonal image zonal stats."""
    #print("date: ", date_)
    year = str(date_[:4])
    month = str(date_[4:])
    #print("month: ", month)
    
    month_, day_range = monthrange(int(year), int(month))
    end_date = str(year) + str(month) + str(day_range)
    #print(end_date)
    return end_date


def im_date_season(df):
    """Collate start date of image into im_date column"""
    
    st_date_list = []
    e_date_list = []
    for i in df.im_name:
        #print(i)
        list_name = i.split("_")
        date = list_name[-2]
        st_date = date[1:7]
        start_date = start_seasonal_date(st_date)
        st_date_list.append(start_date)
        
        e_date = date[7:] 
        end_date = end_seasonal_date(e_date)
        e_date_list.append(end_date)
        
    df["im_s_date"] = st_date_list
    df["im_e_date"] = e_date_list
    
    return df
        
    
def im_date_annual(df):
    """Collate start date of image into im_date column"""
    
    st_date_list = []
    e_date_list = []
    for i in df.im_name:
        #print(i)
        list_name = i.split("_")
        date = list_name[-2]
        st_date = str(date) + "01"
        start_date = start_seasonal_date(st_date)
        st_date_list.append(start_date)
        
        e_date = str(date) + "12"
        end_date = end_seasonal_date(e_date)
        e_date_list.append(end_date)
        
    df["im_s_date"] = st_date_list
    df["im_e_date"] = e_date_list
    
    return df

    
def convert_to_datetime(df, col_nm_s, col_nm_d):
    
    date_list = []
    for i in df[col_nm_s]:
        #print(i)
        datetime_object = datetime.strptime(str(i), '%Y%m%d')
        date_list.append(datetime_object)
        print(datetime_object)
        #df[col_nm_d] =  pd.to_datetime(df[col_nm_s], format='%Y%m%d.%f')
        #date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
    df[col_nm_d] = date_list
    return df        

In [6]:
basal_df = convert_to_datetime(basal_df, "date", "date_d")
basal_df.sort_values(by='date_d', inplace=True)

2012-06-05 00:00:00
2012-05-22 00:00:00
2012-05-23 00:00:00
2012-05-23 00:00:00
2012-10-10 00:00:00
2012-10-10 00:00:00
2012-10-11 00:00:00
2012-10-11 00:00:00
2012-10-11 00:00:00
2013-04-26 00:00:00
2012-07-13 00:00:00
2012-07-13 00:00:00
2012-06-05 00:00:00
2012-06-06 00:00:00
2012-07-10 00:00:00
2012-07-10 00:00:00
2012-06-06 00:00:00
2012-07-10 00:00:00
2012-10-05 00:00:00
2012-10-05 00:00:00
2012-10-06 00:00:00
2012-10-07 00:00:00
2012-10-07 00:00:00
2012-10-07 00:00:00
2012-10-08 00:00:00
2012-10-08 00:00:00
2012-10-09 00:00:00
2012-10-10 00:00:00
2013-06-02 00:00:00
2012-06-02 00:00:00
2013-07-16 00:00:00


In [7]:
sub_list = next(os.walk(dir_))[1]

In [8]:
# zonal_dir = []
# for sub_dir in sub_list:
#     file_list = []
#     if "zonal_stats" in sub_dir:
#         for file_ in glob(os.path.join(dir_, sub_dir, "*.csv")):
#             print(file_)
#             df = pd.read_csv(file_)
#             file_list.append(df)
#         df1 = pd.concat(file_list)
#         zonal_dir.append(df1)

In [9]:
zonal_list = []
sub_dir_list = []
for sub_dir in sub_list:
    file_list = []
    if "zonal_stats" in sub_dir:
        sub_dir_list.append(sub_dir)
        print(sub_dir)
        for file_ in glob(os.path.join(dir_, sub_dir, "*.csv")):
            #print(file_)
            df = pd.read_csv(file_)
            file_list.append(df)
        if len(file_list) > 1:
            df1 = pd.concat(file_list)
            
            # delete column im_date
            if "im_date" in df1.columns:
                df1.drop(['im_date'], axis=1, inplace=True)
                
                
            df1.reset_index(inplace=True, drop=True)
            #print(df.columns)
            
            data = dict_.get(sub_dir) #, default = None)
            
            # set seaonal rule set due to date differences in file name  
            print("seasonal_Type: ", data)
            
            if data == "seasonal":
                df1 =im_date_season(df1)
            elif data == "annual":
                df1 =im_date_annual(df1)
            elif data == "unknown":
                print("Unknown date format add info to dict_")
                import sys
                sys.exit()
            else:
                print("Unknown date format add info to dict_")
                import sys
                sys.exit()
                
            # convert to datetime
            df1 = convert_to_datetime(df1, "im_s_date", "im_s_d")
            df1 = convert_to_datetime(df1, "im_e_date", "im_e_d")
            #sort df on datetime data
            df1.sort_values(by='im_s_date', inplace=True)
            #merge data with basal datset based on the nearest date to the field data colection
            df2 = pd.merge_asof(basal_df, df1, left_on="date_d", right_on= "im_s_d", by="site", direction="nearest")
            
            df2.to_csv(os.path.join(output_dir, "merged_basal_zonal", "merged_biomass_{0}.csv".format(sub_dir)), index=False)
            
            
            #pg_all_df = pd.merge_asof(basal_df, df3, left_on="date_d", right_on = "im_s_d", by="site", direction="nearest")


            #df2 = pd.merge(basal_df, df1, left_on='site', right_on='site')
            #df2.to_csv(os.path.join(output_dir, "concat_{0}.csv".format(sub_dir)))
            
            #df2 = pd.concat([basal_df, df1], axis=1)
            zonal_list.append(df2)
        else:
            pass

dbi_zonal_stats
seasonal_Type:  seasonal
2021-06-01 00:00:00
2021-09-01 00:00:00
2021-12-01 00:00:00
2022-03-01 00:00:00
2021-06-01 00:00:00
2021-09-01 00:00:00
2021-12-01 00:00:00
2022-03-01 00:00:00
2021-06-01 00:00:00
2021-09-01 00:00:00
2021-12-01 00:00:00
2022-03-01 00:00:00
2021-06-01 00:00:00
2021-09-01 00:00:00
2021-12-01 00:00:00
2022-03-01 00:00:00
2021-06-01 00:00:00
2021-09-01 00:00:00
2021-12-01 00:00:00
2022-03-01 00:00:00
2021-06-01 00:00:00
2021-09-01 00:00:00
2021-12-01 00:00:00
2022-03-01 00:00:00
2021-06-01 00:00:00
2021-09-01 00:00:00
2021-12-01 00:00:00
2022-03-01 00:00:00
2021-06-01 00:00:00
2021-09-01 00:00:00
2021-12-01 00:00:00
2022-03-01 00:00:00
2021-06-01 00:00:00
2021-09-01 00:00:00
2021-12-01 00:00:00
2022-03-01 00:00:00
2021-06-01 00:00:00
2021-09-01 00:00:00
2021-12-01 00:00:00
2022-03-01 00:00:00
2021-06-01 00:00:00
2021-09-01 00:00:00
2021-12-01 00:00:00
2022-03-01 00:00:00
2021-06-01 00:00:00
2021-09-01 00:00:00
2021-12-01 00:00:00
2022-03-01 00:00:00

2016-09-01 00:00:00
2016-12-01 00:00:00
2017-03-01 00:00:00
2017-06-01 00:00:00
2017-09-01 00:00:00
2017-12-01 00:00:00
2018-03-01 00:00:00
2018-06-01 00:00:00
2018-09-01 00:00:00
2018-12-01 00:00:00
2019-03-01 00:00:00
2019-06-01 00:00:00
2019-09-01 00:00:00
2019-12-01 00:00:00
2020-03-01 00:00:00
2020-06-01 00:00:00
2020-09-01 00:00:00
2020-12-01 00:00:00
2021-03-01 00:00:00
2021-06-01 00:00:00
2021-09-01 00:00:00
1987-03-01 00:00:00
1987-06-01 00:00:00
1987-09-01 00:00:00
1987-12-01 00:00:00
1988-03-01 00:00:00
1988-06-01 00:00:00
1988-09-01 00:00:00
1988-12-01 00:00:00
1989-03-01 00:00:00
1989-06-01 00:00:00
1989-09-01 00:00:00
1989-12-01 00:00:00
1990-03-01 00:00:00
1990-06-01 00:00:00
1990-09-01 00:00:00
1990-12-01 00:00:00
1991-03-01 00:00:00
1991-06-01 00:00:00
1991-09-01 00:00:00
1991-12-01 00:00:00
1992-03-01 00:00:00
1992-06-01 00:00:00
1992-09-01 00:00:00
1992-12-01 00:00:00
1993-03-01 00:00:00
1993-06-01 00:00:00
1993-09-01 00:00:00
1993-12-01 00:00:00
1994-03-01 00:00:00


dis_zonal_stats
seasonal_Type:  seasonal
1987-06-01 00:00:00
1987-09-01 00:00:00
1987-12-01 00:00:00
1988-03-01 00:00:00
1988-06-01 00:00:00
1988-09-01 00:00:00
1988-12-01 00:00:00
1989-03-01 00:00:00
1989-06-01 00:00:00
1989-09-01 00:00:00
1989-12-01 00:00:00
1990-03-01 00:00:00
1990-06-01 00:00:00
1990-09-01 00:00:00
1990-12-01 00:00:00
1991-03-01 00:00:00
1991-06-01 00:00:00
1991-09-01 00:00:00
1991-12-01 00:00:00
1992-03-01 00:00:00
1992-06-01 00:00:00
1992-09-01 00:00:00
1992-12-01 00:00:00
1993-03-01 00:00:00
1993-06-01 00:00:00
1993-09-01 00:00:00
1993-12-01 00:00:00
1994-03-01 00:00:00
1994-06-01 00:00:00
1994-09-01 00:00:00
1994-12-01 00:00:00
1995-03-01 00:00:00
1995-06-01 00:00:00
1995-09-01 00:00:00
1995-12-01 00:00:00
1996-03-01 00:00:00
1996-06-01 00:00:00
1996-09-01 00:00:00
1996-12-01 00:00:00
1997-03-01 00:00:00
1997-06-01 00:00:00
1997-09-01 00:00:00
1997-12-01 00:00:00
1998-03-01 00:00:00
1998-06-01 00:00:00
1998-09-01 00:00:00
1998-12-01 00:00:00
1999-03-01 00:00:00

2021-03-01 00:00:00
2021-06-01 00:00:00
2021-09-01 00:00:00
2021-12-01 00:00:00
1987-06-01 00:00:00
1987-09-01 00:00:00
1987-12-01 00:00:00
1988-03-01 00:00:00
1988-06-01 00:00:00
1988-09-01 00:00:00
1988-12-01 00:00:00
1989-03-01 00:00:00
1989-06-01 00:00:00
1989-09-01 00:00:00
1989-12-01 00:00:00
1990-03-01 00:00:00
1990-06-01 00:00:00
1990-09-01 00:00:00
1990-12-01 00:00:00
1991-03-01 00:00:00
1991-06-01 00:00:00
1991-09-01 00:00:00
1991-12-01 00:00:00
1992-03-01 00:00:00
1992-06-01 00:00:00
1992-09-01 00:00:00
1992-12-01 00:00:00
1993-03-01 00:00:00
1993-06-01 00:00:00
1993-09-01 00:00:00
1993-12-01 00:00:00
1994-03-01 00:00:00
1994-06-01 00:00:00
1994-09-01 00:00:00
1994-12-01 00:00:00
1995-03-01 00:00:00
1995-06-01 00:00:00
1995-09-01 00:00:00
1995-12-01 00:00:00
1996-03-01 00:00:00
1996-06-01 00:00:00
1996-09-01 00:00:00
1996-12-01 00:00:00
1997-03-01 00:00:00
1997-06-01 00:00:00
1997-09-01 00:00:00
1997-12-01 00:00:00
1998-03-01 00:00:00
1998-06-01 00:00:00
1998-09-01 00:00:00


dja_zonal_stats
seasonal_Type:  seasonal
1989-12-01 00:00:00
1990-03-01 00:00:00
1990-06-01 00:00:00
1990-09-01 00:00:00
1990-12-01 00:00:00
1991-03-01 00:00:00
1991-06-01 00:00:00
1991-09-01 00:00:00
1991-12-01 00:00:00
1992-03-01 00:00:00
1992-06-01 00:00:00
1992-09-01 00:00:00
1992-12-01 00:00:00
1993-03-01 00:00:00
1993-06-01 00:00:00
1993-09-01 00:00:00
1993-12-01 00:00:00
1994-03-01 00:00:00
1994-06-01 00:00:00
1994-09-01 00:00:00
1994-12-01 00:00:00
1995-03-01 00:00:00
1995-06-01 00:00:00
1995-09-01 00:00:00
1995-12-01 00:00:00
1996-03-01 00:00:00
1996-06-01 00:00:00
1996-09-01 00:00:00
1996-12-01 00:00:00
1997-03-01 00:00:00
1997-06-01 00:00:00
1997-09-01 00:00:00
1997-12-01 00:00:00
1998-03-01 00:00:00
1998-06-01 00:00:00
1998-09-01 00:00:00
1998-12-01 00:00:00
1999-03-01 00:00:00
1999-06-01 00:00:00
1999-09-01 00:00:00
1999-12-01 00:00:00
2000-03-01 00:00:00
2000-06-01 00:00:00
2000-09-01 00:00:00
2000-12-01 00:00:00
2001-03-01 00:00:00
2001-06-01 00:00:00
2001-09-01 00:00:00

1991-11-30 00:00:00
1992-02-29 00:00:00
1992-05-31 00:00:00
1992-08-31 00:00:00
1992-11-30 00:00:00
1993-02-28 00:00:00
1993-05-31 00:00:00
1993-08-31 00:00:00
1993-11-30 00:00:00
1994-02-28 00:00:00
1994-05-31 00:00:00
1994-08-31 00:00:00
1994-11-30 00:00:00
1995-02-28 00:00:00
1995-05-31 00:00:00
1995-08-31 00:00:00
1995-11-30 00:00:00
1996-02-29 00:00:00
1996-05-31 00:00:00
1996-08-31 00:00:00
1996-11-30 00:00:00
1997-02-28 00:00:00
1997-05-31 00:00:00
1997-08-31 00:00:00
1997-11-30 00:00:00
1998-02-28 00:00:00
1998-05-31 00:00:00
1998-08-31 00:00:00
1998-11-30 00:00:00
1999-02-28 00:00:00
1999-05-31 00:00:00
1999-08-31 00:00:00
1999-11-30 00:00:00
2000-02-29 00:00:00
2000-05-31 00:00:00
2000-08-31 00:00:00
2000-11-30 00:00:00
2001-02-28 00:00:00
2001-05-31 00:00:00
2001-08-31 00:00:00
2001-11-30 00:00:00
2002-02-28 00:00:00
2002-05-31 00:00:00
2002-08-31 00:00:00
2002-11-30 00:00:00
2003-02-28 00:00:00
2003-05-31 00:00:00
2003-08-31 00:00:00
2003-11-30 00:00:00
2004-02-29 00:00:00


1997-12-31 00:00:00
1998-12-31 00:00:00
1999-12-31 00:00:00
2000-12-31 00:00:00
2001-12-31 00:00:00
2002-12-31 00:00:00
2003-12-31 00:00:00
2004-12-31 00:00:00
2005-12-31 00:00:00
2006-12-31 00:00:00
2007-12-31 00:00:00
2008-12-31 00:00:00
2009-12-31 00:00:00
2010-12-31 00:00:00
2011-12-31 00:00:00
2012-12-31 00:00:00
2013-12-31 00:00:00
2014-12-31 00:00:00
2015-12-31 00:00:00
2016-12-31 00:00:00
2017-12-31 00:00:00
1987-12-31 00:00:00
1988-12-31 00:00:00
1989-12-31 00:00:00
1990-12-31 00:00:00
1991-12-31 00:00:00
1992-12-31 00:00:00
1993-12-31 00:00:00
1994-12-31 00:00:00
1995-12-31 00:00:00
1996-12-31 00:00:00
1997-12-31 00:00:00
1998-12-31 00:00:00
1999-12-31 00:00:00
2000-12-31 00:00:00
2001-12-31 00:00:00
2002-12-31 00:00:00
2003-12-31 00:00:00
2004-12-31 00:00:00
2005-12-31 00:00:00
2006-12-31 00:00:00
2007-12-31 00:00:00
2008-12-31 00:00:00
2009-12-31 00:00:00
2010-12-31 00:00:00
2011-12-31 00:00:00
2012-12-31 00:00:00
2013-12-31 00:00:00
2014-12-31 00:00:00
2015-12-31 00:00:00


2014-01-01 00:00:00
2015-01-01 00:00:00
2016-01-01 00:00:00
2017-01-01 00:00:00
2018-01-01 00:00:00
2019-01-01 00:00:00
2020-01-01 00:00:00
2021-05-01 00:00:00
1988-01-01 00:00:00
1989-01-01 00:00:00
1990-01-01 00:00:00
1991-01-01 00:00:00
1992-01-01 00:00:00
1993-01-01 00:00:00
1994-01-01 00:00:00
1995-01-01 00:00:00
1996-01-01 00:00:00
1997-01-01 00:00:00
1998-01-01 00:00:00
1999-01-01 00:00:00
2000-01-01 00:00:00
2001-01-01 00:00:00
2002-01-01 00:00:00
2003-01-01 00:00:00
2004-01-01 00:00:00
2005-01-01 00:00:00
2006-01-01 00:00:00
2007-01-01 00:00:00
2008-01-01 00:00:00
2009-01-01 00:00:00
2010-01-01 00:00:00
2011-01-01 00:00:00
2012-01-01 00:00:00
2013-01-01 00:00:00
2014-01-01 00:00:00
2015-01-01 00:00:00
2016-01-01 00:00:00
2017-01-01 00:00:00
2018-01-01 00:00:00
2019-01-01 00:00:00
2020-01-01 00:00:00
2021-05-01 00:00:00
1988-01-01 00:00:00
1989-01-01 00:00:00
1990-01-01 00:00:00
1991-01-01 00:00:00
1992-01-01 00:00:00
1993-01-01 00:00:00
1994-01-01 00:00:00
1995-01-01 00:00:00


2006-01-01 00:00:00
2007-01-01 00:00:00
2008-01-01 00:00:00
2009-01-01 00:00:00
2010-01-01 00:00:00
2011-01-01 00:00:00
2012-01-01 00:00:00
2013-01-01 00:00:00
2014-01-01 00:00:00
2015-01-01 00:00:00
2016-01-01 00:00:00
2017-01-01 00:00:00
2018-01-01 00:00:00
2019-01-01 00:00:00
2020-01-01 00:00:00
2021-01-01 00:00:00
1988-01-01 00:00:00
1989-01-01 00:00:00
1990-01-01 00:00:00
1991-01-01 00:00:00
1992-01-01 00:00:00
1993-01-01 00:00:00
1994-01-01 00:00:00
1995-01-01 00:00:00
1996-01-01 00:00:00
1997-01-01 00:00:00
1998-01-01 00:00:00
1999-01-01 00:00:00
2000-01-01 00:00:00
2001-01-01 00:00:00
2002-01-01 00:00:00
2003-01-01 00:00:00
2004-01-01 00:00:00
2005-01-01 00:00:00
2006-01-01 00:00:00
2007-01-01 00:00:00
2008-01-01 00:00:00
2009-01-01 00:00:00
2010-01-01 00:00:00
2011-01-01 00:00:00
2012-01-01 00:00:00
2013-01-01 00:00:00
2014-01-01 00:00:00
2015-01-01 00:00:00
2016-01-01 00:00:00
2017-01-01 00:00:00
2018-01-01 00:00:00
2019-01-01 00:00:00
2020-01-01 00:00:00
2021-01-01 00:00:00
